In [1]:
import sys, platform, site

print("Python:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("Machine:", platform.machine())
print("Site-packages:", site.getsitepackages())

Python: 3.11.14 (main, Oct  9 2025, 16:16:55) [Clang 17.0.0 (clang-1700.4.4.1)]
Executable: /Users/mariaburtseva/Documents/проект грант/slr-env/.venv/bin/python
Platform: macOS-26.2-arm64-arm-64bit
Machine: arm64
Site-packages: ['/Users/mariaburtseva/Documents/проект грант/slr-env/.venv/lib/python3.11/site-packages']


In [2]:
import torch

print("torch:", torch.__version__)
mps_available = torch.backends.mps.is_available()
mps_built = torch.backends.mps.is_built()
print("MPS built:", mps_built)
print("MPS available:", mps_available)

device = "mps" if mps_available else "cpu"
print("Using device:", device)

x = torch.randn(1024, 1024, device=device)
y = x @ x.T
print("OK, y mean:", y.mean().item())

torch: 2.9.1
MPS built: True
MPS available: True
Using device: mps
OK, y mean: 0.8867493867874146


In [2]:
import numpy as np
import mediapipe as mp

print("mediapipe:", mp.__version__)

# тестовая картинка (чёрная) — результат может быть пустой, это ОК
rgb = np.zeros((480, 640, 3), dtype=np.uint8)

mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path="models/hand_landmarker.task"),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2,
    min_hand_detection_confidence=0.5
)

with HandLandmarker.create_from_options(options) as landmarker:
    result = landmarker.detect(mp_image)

print("Hands found:", len(result.hand_landmarks))



mediapipe: 0.10.31
Hands found: 0


I0000 00:00:1768216289.974359 1074148 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4
W0000 00:00:1768216289.979771 1074150 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768216289.983868 1074150 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
import time
import cv2
import mediapipe as mp
from pathlib import Path

# --- resolve project root (where .git lives), so the notebook can run from anywhere ---
root = Path.cwd()
while root != root.parent and not (root / ".git").exists():
    root = root.parent

MODEL_PATH = root / "models" / "hand_landmarker.task"
print("Resolved MODEL_PATH:", MODEL_PATH)

if not MODEL_PATH.exists():
    raise FileNotFoundError(
        f"Model file not found: {MODEL_PATH}\n"
        "Скачай модель в корень репозитория:\n"
        "mkdir -p models\n"
        "curl -L -o models/hand_landmarker.task "
        "https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/latest/hand_landmarker.task"
    )

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=str(MODEL_PATH)),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=2,
)

WIN = "MediaPipe HandLandmarker (q/ESC to quit)"
cv2.namedWindow(WIN, cv2.WINDOW_NORMAL)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Камера не открылась. Попробуй VideoCapture(1).")

start = time.time()

try:
    with HandLandmarker.create_from_options(options) as landmarker:
        while True:
            ok, frame = cap.read()
            if not ok:
                print("Не удалось прочитать кадр с камеры")
                break

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

            timestamp_ms = int((time.time() - start) * 1000)
            result = landmarker.detect_for_video(mp_image, timestamp_ms)

            if result.hand_landmarks:
                h, w = frame.shape[:2]
                for hand in result.hand_landmarks:
                    for lm in hand:
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)

            cv2.imshow(WIN, frame)

            # если окно закрыли крестиком — выходим до следующего imshow
            if cv2.getWindowProperty(WIN, cv2.WND_PROP_VISIBLE) < 1:
                break

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q") or key == 27:
                break
finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Camera test done")

Resolved MODEL_PATH: /Users/mariaburtseva/Documents/проект грант/slr-env/models/hand_landmarker.task


I0000 00:00:1768216145.532533 1072328 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1768216145.555413 1072332 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768216145.567515 1072332 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768216145.596680 1072334 landmark_projection_calculator.cc:81] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Camera test done


: 